In [ ]:
from datetime import date
import pandas as pd
import polars as pl
import numpy as np
import streamlit as st
import plotly.graph_objects as go






In [62]:
leistung_wr

,Datetime,wr,string,sensor,value,standort
0,2025-11-22 00:00:00,1,-1,P,0,badboll
1,2025-11-22 00:00:00,2,-1,P,0,badboll
2,2025-11-22 00:00:00,3,-1,P,0,badboll
3,2025-11-22 00:05:00,1,-1,P,0,badboll
4,2025-11-22 00:05:00,2,-1,P,0,badboll
...,...,...,...,...,...,...
571,2025-11-22 16:55:00,2,-1,P,0,badboll
572,2025-11-22 16:55:00,3,-1,P,0,badboll
573,2025-11-22 17:00:00,1,-1,P,0,badboll
574,2025-11-22 17:00:00,2,-1,P,0,badboll


In [ ]:
from datetime import date
import pandas as pd
import polars as pl
import numpy as np
import plotly.graph_objects as go
import pvlib
import pytz

from backend_leistung import get_day_and_update
# ------------------------------------------
# CONFIG / PARAMETER
# ------------------------------------------
LAT, LON, TZ = 48.39, 9.36, "Europe/Berlin"
ALTITUDE = 411
STANDORT = "muensingen"
FILE_PATH = "app/data/leistung.parquet"

TILT = 30.0                # PV-Modul Neigung (°)
AZIMUTH =155.0             # PV-Ausrichtung (°) – 180 = Süden

# Farben / Styling
BASE_COLOR = "#FFCC00"
NUM_LAYERS = 6
ALPHAS = np.linspace(0.03, 0.30, NUM_LAYERS)
WR_DASH = ['dot', 'dash', 'dashdot', 'longdashdot', 'dot', 'dash', 'dashdot']

# ------------------------------------------
# FUNKTIONEN
# ------------------------------------------
def load_pv_data(file_path: str, standort: str, datum: date):
    """Lade PV-Leistungsdaten aus Parquet-Datei und aggregiere Gesamtleistung."""
   # df_polars = pl.scan_parquet(file_path)
    df_polars = pl.LazyFrame(get_day_and_update(standort,datum))
    # Gesamtleistung je Zeitpunkt
    leistung_gesamt = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == datum) &
            (pl.col("string") == -1) &
            (pl.col("sensor") == "P")
        )
        .group_by("Datetime")
        .agg(pl.col("value").sum().alias("P_gesamt"))
        .sort("Datetime")
        .collect(engine="streaming")
        .to_pandas()
    )
    
    # Einzel-WR-Leistung
    leistung_wr = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == datum) &
            (pl.col("string") == -1) &
            (pl.col("sensor") == "P")
        )
        .sort("Datetime")
        .collect(engine="streaming")
        .to_pandas()
    )
    
    return leistung_gesamt, leistung_wr

def get_sun_times(lat: float, lon: float, tz: str, datum: pd.Timestamp):
    """Berechne Sonnenaufgang und -untergang."""
    location = pvlib.location.Location(lat, lon, tz=tz, altitude=ALTITUDE)
    times_for_sun = pd.DatetimeIndex([datum + pd.Timedelta(hours=12)], tz=tz)
    sun_df = location.get_sun_rise_set_transit(times_for_sun, method='spa')
    return sun_df['sunrise'].iloc[0], sun_df['sunset'].iloc[0]

def compute_clearsky_pv(lat: float, lon: float, tz: str, times: pd.DatetimeIndex,
                        tilt: float, azimuth: float, eff: float):
    """Berechne theoretische maximale PV-Leistung (Clearsky)."""
    location = pvlib.location.Location(lat, lon, tz=tz, altitude=ALTITUDE)
    clearsky = location.get_clearsky(times, model='ineichen')
    solarpos = location.get_solarposition(times)
    
    dni_extra = pvlib.irradiance.get_extra_radiation(times)
    poa = pvlib.irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        solar_zenith=solarpos['apparent_zenith'],
        solar_azimuth=solarpos['azimuth'],
        dni=clearsky['dni'],
        ghi=clearsky['ghi'],
        dhi=clearsky['dhi'],
        dni_extra=dni_extra,
        model='haydavies'
    )
    
    p_dc_simple = poa['poa_global']  * eff
    return p_dc_simple

def create_pv_plot(leistung_gesamt, leistung_wr, p_dc_simple, sunrise, sunset,df):
    """Erzeuge Plotly-Figure mit Gesamt- und WR-Leistung + Clearsky."""
    x = pd.to_datetime(leistung_gesamt["Datetime"])
    y_kw = leistung_gesamt["P_gesamt"].to_numpy() / 1000.0

    fig = go.Figure()

    # Gefüllte Layers
    for i in range(NUM_LAYERS):
        frac = (i + 1) / NUM_LAYERS
        y_layer = y_kw * frac
        fill_mode = "tozeroy" if i == 0 else "tonexty"
        rgba = f"rgba(255,204,0,{ALPHAS[i]:.3f})"
        fig.add_trace(go.Scatter(
            x=x, y=y_layer, mode="lines", line=dict(width=0),
            fill=fill_mode, fillcolor=rgba, hoverinfo="skip", showlegend=False
        ))

    # Gesamtleistung
    fig.add_trace(go.Scatter(
        x=x, y=y_kw, mode="lines", line=dict(color=BASE_COLOR, width=4),
        name="Gesamt", hovertemplate="%{x|%H:%M}: %{y:.2f} kW"
    ))

    # Einzel-WR-Leistung
    for i, wr in enumerate(leistung_wr["wr"].unique()):
        temp_x = pd.to_datetime(leistung_wr.loc[leistung_wr["wr"]==wr]["Datetime"])
        temp_y = leistung_wr.loc[leistung_wr["wr"]==wr]["value"] / 1_000
        fig.add_trace(go.Scatter(
            x=temp_x, y=temp_y, mode="lines",
            line=dict(color="#365FB7", width=2, dash=WR_DASH[i % len(WR_DASH)]),
            name=f"Wechselrichter {wr}",
            hovertemplate="%{x|%H:%M}: %{y:.2f} kW"
        ))

    # Clearsky / theoretisches Maximum
    # fig.add_trace(go.Scatter(
    #     x=p_dc_simple.index,
    #     y=p_dc_simple/1000,
    #     mode="lines",
    #     line=dict(color="orange", width=2, dash="dot"),
    #     name="Clearsky Max"
    # ))
    
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df["Watts"]/1000,
        mode="lines",
        line=dict(color="orange", width=2, dash="dot"),
        name="Vorhersage"
    ))

    # Layout
    fig.update_layout(
        template="simple_white",
        margin=dict(l=60, r=20, t=30, b=60),
        xaxis=dict(
            title="Uhrzeit",
            tickformat="%H:%M",
            showgrid=False,
            showline=True,
            linewidth=1,
            linecolor="#cccccc",
            range=[sunrise, sunset],
            showspikes=True,
            spikecolor="gray",
            spikethickness=3,
            spikesnap="cursor",
            spikemode="across"
        ),
        yaxis=dict(title="Leistung (kW)", showgrid=True, gridcolor="#f2f2f2", zeroline=False),
        hovermode="x unified",
        hoverlabel=dict(bgcolor="white", bordercolor="gray", font_size=14, font_family="Arial", align="left", namelength=-1)
    )
    
    return fig

# ------------------------------------------
# MAIN
# ------------------------------------------

heute = date.today()
heute = pd.Timestamp("2025-11-29", tz=TZ).date()
leistung_gesamt, leistung_wr = load_pv_data(FILE_PATH, STANDORT, heute)

sunrise, sunset = get_sun_times(LAT, LON, TZ, pd.Timestamp(heute, tz=TZ))

times = pd.date_range(start=sunrise.floor('min'), end=sunset.ceil('min'), freq='1min', tz=TZ)

EFFICENCY=40
p_dc_simple = compute_clearsky_pv(LAT, LON, TZ, times, TILT, AZIMUTH, EFFICENCY)

fig = create_pv_plot(leistung_gesamt, leistung_wr, p_dc_simple, sunrise, sunset)
fig.show()

[getDay]: muensingen - 2025-11-29 - heute... Downloaded shape: (5754, 6)


/Users/simon/Desktop/OEEGDashboard/.venv/lib/python3.12/site-packages/plotly/io/_json.py:558: UserWarning:

Discarding nonzero nanoseconds in conversion.



In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go

lat = 48.411 # latitude of location, -90 (south) … 90 (north);
lon = 9.498 #longitude of location, -180 (west) … 180 (east)
dec = 30.0 # plane declination, 0 (horizontal) … 90 (vertical) 
az = "S" # plane azimuth, -180 … 180 (-180 = north, -90 = east, 0 = south, 90 = west, 180 = north)
kwp=75.9

def get_forecast(lat, lon, dec, az, kwp):
    # send request to https://api.forecast.solar/estimate/:lat/:lon/:dec/:az/:kwp
    
    url = f"https://api.forecast.solar/estimate/{lat}/{lon}/{dec}/{az}/{kwp}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(response)
        return None

forecast = get_forecast(lat, lon, dec, az, kwp)
# forecast["result"]["watts"] is a dictionary to dataframe
df: pd.DataFrame = pd.DataFrame.from_dict(forecast["result"]["watts"], orient="index", columns=["Watts"])

#%%
# plotly scatter plot using plotly go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["Watts"], mode="lines", name="Forecasted Watts"))
fig.update_layout(title="Solar Power Forecast", xaxis_title="Time", yaxis_title="Watts")
fig.show()


In [2]:
from update_leistungsdaten import update_leistung
import pandas as pd
update_leistung(end=pd.Timestamp("2020-07-20", tz=TZ).date())

NameError: name 'TZ' is not defined

In [26]:
import pandas as pd
import pvlib
import numpy as np

# Standort
lat, lon, tz = 48.39, 9.36, "Europe/Berlin"
location = pvlib.location.Location(lat, lon, tz=tz)

# Datum (heute) und Zeitraum zwischen Sonnenaufgang und -untergang
today = pd.Timestamp('now', tz=tz).normalize()
# Nutze Location, um Sunrise/Sunset zuverlässig zu bekommen:
times_for_sun = pd.DatetimeIndex([today + pd.Timedelta(hours=12)], tz=tz)  # local noon
sun_df = location.get_sun_rise_set_transit(times_for_sun, method='spa')
sunrise = sun_df['sunrise'].iloc[0]
sunset  = sun_df['sunset'].iloc[0]

# Zeitauflösung für Tagesverlauf
times = pd.date_range(start=sunrise.floor('min'), end=sunset.ceil('min'), freq='1min', tz=tz)

# Clearsky (theoretisches Maximum) - Modell: "ineichen" ist eine robuste Wahl
clearsky = location.get_clearsky(times, model='ineichen')  # gibt 'ghi','dni','dhi'

# Solarposition für POA-Berechnung
solarpos = location.get_solarposition(times)

# Anlagengeometrie / Annahmen (Bitte anpassen)
tilt = 30.0                # Neigung (°)
azimuth = 180.0            # Ausrichtung (°) — 180 = Süden
module_area_m2 = 1.94      # Fläche eines Moduls in m² (Beispiel)
n_modules = 43             # Anzahl Module => array area = module_area_m2 * n_modules
array_area = module_area_m2 * n_modules
assumed_module_eff = 0.18  # 18% Wirkungsgrad bei STC, stell hier deine Zahl ein

# Berechne POA Global Irradiance (W/m²)
dni_extra = pvlib.irradiance.get_extra_radiation(times)
poa = pvlib.irradiance.get_total_irradiance(
    surface_tilt=tilt,
    surface_azimuth=azimuth,
    solar_zenith=solarpos['apparent_zenith'],
    solar_azimuth=solarpos['azimuth'],
    dni=clearsky['dni'],
    ghi=clearsky['ghi'],
    dhi=clearsky['dhi'],
    dni_extra=dni_extra,
    model='haydavies'   # beliebter Diffusmodell-Choice; kannst ändern
)
poa_global = poa['poa_global']  # W/m²

# Vereinfachte DC-Leistungsabschätzung (ohne Temperatur- oder Wirkungsgradverluste)
p_dc_simple = poa_global * array_area * assumed_module_eff  # in Watt (W)

# Peak-Wert (theoretisches Maximum an diesem Tag)
p_dc_peak_w = p_dc_simple.max()
p_dc_peak_kw = p_dc_peak_w / 1000.0

print(f"Theoretische Peak-DC (clearsky, simple): {p_dc_peak_kw:.2f} kW")

Theoretische Peak-DC (clearsky, simple): 9.76 kW


In [27]:
p_dc_simple

2025-11-29 07:50:00+01:00     0.000000
2025-11-29 07:51:00+01:00     0.000000
2025-11-29 07:52:00+01:00     0.000000
2025-11-29 07:53:00+01:00     9.435556
2025-11-29 07:54:00+01:00    12.971669
                               ...    
2025-11-29 16:28:00+01:00    11.199943
2025-11-29 16:29:00+01:00     0.000000
2025-11-29 16:30:00+01:00     0.000000
2025-11-29 16:31:00+01:00     0.000000
2025-11-29 16:32:00+01:00     0.000000
Freq: min, Name: poa_global, Length: 523, dtype: float64

In [28]:
## Barchart trace
bar_trace = go.Bar(x = ['Bar 1', 'Bar 2', 'Bar 3'], y = [20, 15, 18])
## Scatter trace one
scatter_trace1 = go.Scatter({'x':[1, 2, 3, 4, 5], 'y':[10, 5, 8, 2, 7], 
'mode':'lines+markers+text','text':["Start", "B", "C", "D", "E"], 
'textposition':'top center'})
## Scatter trace two
scatter_trace2 = go.Scatter({'x':[1, 2, 3, 4, 5], 'y':[10,13,15,17,16], 
'mode':'lines+markers+text','text':["Start", "2", "3", "4", "5"], 
'textposition':'top center'})

In [3]:
from ui_calendar import plot_calendar_heatmap
import pandas as pd


ertrag = pd.read_parquet("app/data/ertrag.parquet")
#column date to 2010-03-23 to datetime
ertrag["date"] = pd.to_datetime(ertrag["date"]).dt.date
ertrag = ertrag.loc[ertrag.standort=="muensingen"]
#select only last year
ertrag = ertrag.loc[ertrag["date"] >= pd.to_datetime("2025-01-01").date()]
ertrag = ertrag.groupby("date").sum().reset_index()
#custom, colorscale with warm yellow tones
Yellows = [[0.0, 'rgb(255, 250, 220)'],[1.0, 'rgb(255, 180, 0)']]
fig_heatmap = plot_calendar_heatmap(ertrag, date_col='date', value_col='value', colorscale=Yellows, title='Calendar Heatmap (Heatmap)', locale_name='de_DE', scale=30,grid_width=4,
            highlight_date=pd.to_datetime("2025-06-21").date())
# add a small rounded box in black to the plot


fig_heatmap.show(config={
    "displayModeBar": False,
    "displaylogo": False,
    "doubleClick": False,
    # "scrollZoom": False,
    "staticPlot": False,
    #"modeBarButtonsToAdd": ["select2d", "lasso2d","pan2d"],
    #"modeBarButtonsToRemove": ["zoom2d",  "autoScale2d", "resetScale2d"]
})
